In [2]:
from src.ssunet.datasets import BinomDataset, ValidationDataset
from src.ssunet.models import Bit2Bit
from src.ssunet.configs import load_config, example_config_path

config = load_config(example_config_path)
data = config.path_config.load_data_only()
validation_data = config.path_config.load_reference_and_ground_truth()
model = Bit2Bit(config.model_config)
train_data = BinomDataset(data, config.data_config, config.split_params)
validation_data = ValidationDataset(validation_data, config.data_config)
train_loader = config.loader_config.loader(train_data)
validation_loader = config.loader_config.loader(validation_data)

# Print the input size
print(f"input_size: {tuple(next(iter(train_loader))[1].shape)}")

# Train the model
trainer = config.train_config.trainer
trainer.fit(model, train_loader, validation_loader)

ZeroDivisionError: integer division or modulo by zero